[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/recommendation/product-recommender/product_recommender.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/recommendation/product-recommender/product_recommender.ipynb)

# Product Recommendation Engine

Learn how to build a product recommendation engine using collaborative filtering and Pinecone.

In this example, we will generate product recommendations for ecommerce customers based on previous orders and trending items. This example covers preparing the vector embeddings, creating and deploying the Pinecone service, writing data to Pinecone, and finally querying Pinecone to receive a ranked list of recommended products.

---

🚨 _Note that running this on CPU is slow! If running on Google Colab you go to **Runtime > Change runtime type > Hardware accelerator > GPU** to switch to GPU._

---

## Data Preparation

In [ ]:
!pip install -qU numpy pandas scipy

**Import Python Libraries**

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools

In [ ]:
!pip install -qU kaggle

In [ ]:
try:
    import kaggle
except OSError as e:
    print(e)

The first time you `import kaggle` you will see an `OSError`, that is because we need to add our Kaggle credentials to the `/root/.kaggle/kaggle.json` file. You can find these credentials on [Kaggle](https://kaggle.com) by accessing your profile in the top-right corner of the page. This will download a `kaggle.json` file which contains your username and secret key. You can enter them below:

In [ ]:
import json

with open('/root/.kaggle/kaggle.json', 'w') as fp:
    fp.write(json.dumps({"username":"YOUR_USERNAME","key":"YOUR_SECRET_KEY"}))

Now we can download the dataset:

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis

This downloads a set of zip files, we extract them like so:

In [ ]:
import zipfile

files = [
    'instacart-market-basket-analysis.zip',
    'order_products__train.csv.zip',
    'order_products__prior.csv.zip',
    'products.csv.zip',
    'orders.csv.zip'
]

for filename in files:
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

Now we can move on to loading the dataset.

**Load the (Example) Instacart Data**

We are going to use the [Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/data) dataset for this task.

The data used throughout this example is a set of files describing customers' orders over time. The main focus is on the *orders.csv* file, where each line represents a relation between a user and the order. In other words, each line has information on *userid* (user who made the order) and *orderid*. Note there is no information about products in this table. Product information related to specific orders is stored in the *order_product__*.csv* dataset.

In [2]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')

order_products = pd.concat([order_products_train, order_products_prior])

**Preparing data for the model**


The Collaborative Filtering model used in this example requires only users’ historical preferences on a set of items. As there is no explicit rating in the data we are using, the purchase quantity can represent a “confidence” in terms of how strong the interaction was between the user and the products.

The dataframe data will store this data and will be the base for the model.

In [3]:
customer_order_products = pd.merge(orders, order_products, how='inner',on='order_id')

# creating a table with "confidences"
data = customer_order_products.groupby(['user_id', 'product_id'])[['order_id']].count().reset_index()
data.columns=["user_id", "product_id", "total_orders"]
data.product_id = data.product_id.astype('int64')

# Create a lookup frame so we can get the product names back in readable form later.
products_lookup = products[['product_id', 'product_name']].drop_duplicates()
products_lookup['product_id'] = products_lookup.product_id.astype('int64')

We will create two prototype users here and add them to our data dataframe. Each user will be buying only a specific product:
- The first user will be buying only **Mineral Water**
- The second user will be buying baby products: **No More Tears Baby Shampoo** and **Baby Wash & Shampoo**

These users will be later used for querying and examination of the model results.

In [4]:
data_new = pd.DataFrame([[data.user_id.max() + 1, 22802, 97],
                         [data.user_id.max() + 2, 26834, 89],
                         [data.user_id.max() + 2, 12590, 77]
                        ], columns=['user_id', 'product_id', 'total_orders'])
data_new

,user_id,product_id,total_orders
0,206210,22802,97
1,206211,26834,89
2,206211,12590,77


In [5]:
data = pd.concat([data, data_new]).reset_index(drop = True)
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


In the next step, we will first extract user and item unique ids, in order to create a CSR (Compressed Sparse Row) matrix. 


In [6]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.total_orders)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((purchases, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((purchases, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

## Implicit Model

In this section we will demonstrate creation and training of a recommender model using the **implicit** library. The recommendation model is based off the algorithms described in the paper [Collaborative Filtering for Implicit Feedback Datasets](https://www.researchgate.net/publication/220765111_Collaborative_Filtering_for_Implicit_Feedback_Datasets) with performance optimizations described in [Applications of the Conjugate Gradient Method for Implicit Feedback Collaborative Filtering](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.6473&rep=rep1&type=pdf).


In [ ]:
!pip install -qU implicit

In [7]:
import implicit
from implicit import evaluation

#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_user_product, train_percentage=0.9)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=50,
                                             num_threads=1)

alpha_val = 15
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/50 [00:00<?, ?it/s]

We will evaluate the model using the inbuilt library function

In [8]:
test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/192835 [00:00<?, ?it/s]

{'precision': 0.27546883229828883,
 'map': 0.0442792123139327,
 'ndcg': 0.14407831615017366,
 'auc': 0.6548804124633053}

This is what item and user factors look like. These vectors will be stored in our vector index later and used for recommendation.

In [9]:
model.item_factors[1:3]

Matrix([[-1.41394716e-02 -7.85469078e-03 -6.01551216e-03 -1.85511466e-02
  -6.40676357e-03 -6.40198728e-03 -9.99970455e-03 -6.54193107e-03
  -1.37126567e-02  2.32426543e-02 -5.20094018e-03  4.09194094e-04
   4.56247712e-03  2.38485597e-02 -2.40435777e-03  1.37668541e-02
   4.93924553e-03  4.07793187e-03  3.79170594e-03 -9.46274493e-03
  -3.37003507e-02  7.73684448e-03 -3.53912963e-03  6.12016162e-03
   6.61744131e-03  6.96701044e-03  1.82978553e-03 -1.95304537e-03
   5.16883971e-04 -5.81818447e-03 -1.21214204e-02  2.82692560e-03
   6.13338128e-03  8.66098388e-04  4.47903853e-03  6.42127078e-03
  -1.13449607e-03  6.67106453e-03 -1.47818150e-02 -1.05382055e-02
  -5.65192988e-03 -7.19359657e-03 -5.67766372e-03  9.10092425e-03
   1.63281581e-03 -1.07810339e-02 -1.59825142e-02  2.09339429e-03
   3.83407023e-04  3.98562806e-05 -2.75215018e-04 -1.28855673e-03
  -2.12697368e-02 -5.48941409e-03  2.44746823e-03 -1.80884488e-02
   7.99474958e-03 -1.63389388e-02 -1.31361966e-03 -1.30685801e-02
   

In [10]:
model.user_factors[1:3]

Matrix([[ 7.72843719e-01 -1.88261974e+00 -1.27751648e-01  5.55098593e-01
   4.16630119e-01 -2.85241336e-01 -2.51213849e-01  2.52275467e-01
  -6.30572319e-01  2.56867218e+00 -2.22353451e-02 -5.78172445e-01
   5.85655496e-02  2.69911575e+00 -1.11881077e+00  2.72573322e-01
   1.02800310e+00  6.58825710e-02  5.10158718e-01  7.06521451e-01
  -1.90171802e+00  2.22235203e+00 -4.53002751e-01 -1.60160914e-01
   1.26648879e+00 -1.77729237e+00 -3.30207407e-01 -8.64066541e-01
  -1.42625049e-01 -3.02950472e-01 -4.74009961e-01  2.72439986e-01
   7.28228986e-01  8.94330084e-01  7.64488697e-01  1.02250576e+00
   1.49344444e+00  7.49285698e-01 -3.64517123e-01  3.44786733e-01
   7.40179420e-01 -1.48939848e-01 -1.77640104e+00 -1.45238030e+00
   2.03178763e+00 -4.96528417e-01 -7.29420125e-01  9.72527504e-01
  -1.51985908e+00 -5.60409091e-02  1.47466981e+00 -4.86016124e-01
  -1.01080215e+00 -1.12702656e+00  3.32015932e-01 -1.14880002e+00
   3.45389396e-01 -2.14261603e+00 -6.07559383e-01 -7.68664241e-01
   

## Configure Pinecone

Install and setup Pinecone

In [ ]:
!pip install -qU pinecone-client

In [11]:
from pinecone import Pinecone, ServerlessSpec

In [12]:
# Load Pinecone API key
api_key = os.getenv("PINECONE_API_KEY") or "d11bd0fa-9723-487b-ae13-7f51b5b22772"
# Set Pinecone environment. Find next to API key in console
env = os.getenv('PINECONE_ENVIRONMENT') or "YOUR_ENV"

# init connection to pinecone
pinecone = Pinecone(api_key=api_key)
spec=ServerlessSpec(
        cloud="aws",
        region="us-west-2",
    )

[Get a Pinecone API key](http://app.pinecone.io/) if you don't have one.

In [13]:
#List all present indexes associated with your key, should be empty on the first run
pinecone.list_indexes().names()

['product-recommender', 'question-answering', 'hybrid-image-search']

**Create an Index**

In [14]:
# Set a name for your index
index_name = 'product-recommender'

In [15]:
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes().names():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100, spec=spec)

**Connect to the new index**

In [16]:
host = pinecone.describe_index(name=index_name).host
index = pinecone.Index(host=host)

## Load Data

Uploading all items (products that one can buy) and displaying some examples of products and their vector representations.


In [ ]:
!pip install -qU torch

In [17]:
import torch

# Get all of the items
all_items_titles = [{'title': title} for title in products_lookup['product_name']]
all_items_ids = [str(product_id) for product_id in products_lookup['product_id']]

# Transform items into factors
items_factors = model.item_factors

device = "cuda" if torch.cuda.is_available() else "cpu"
item_embeddings = items_factors[1:].to_numpy().tolist() if device == "cuda" else items_factors[1:].tolist()

# Prepare item factors for upload
items_to_insert = list(zip(all_items_ids, item_embeddings, all_items_titles))
display(items_to_insert[:2])

[('1',
  [-0.01413947157561779,
   -0.00785469077527523,
   -0.00601551216095686,
   -0.018551146611571312,
   -0.006406763568520546,
   -0.0064019872806966305,
   -0.009999704547226429,
   -0.0065419310703873634,
   -0.013712656684219837,
   0.023242654278874397,
   -0.0052009401842951775,
   0.00040919409366324544,
   0.0045624771155416965,
   0.023848559707403183,
   -0.002404357772320509,
   0.01376685407012701,
   0.004939245525747538,
   0.0040779318660497665,
   0.0037917059380561113,
   -0.009462744928896427,
   -0.03370035067200661,
   0.007736844476312399,
   -0.003539129626005888,
   0.00612016161903739,
   0.0066174413077533245,
   0.0069670104421675205,
   0.0018297855276614428,
   -0.0019530453719198704,
   0.0005168839707039297,
   -0.005818184465169907,
   -0.012121420353651047,
   0.0028269255999475718,
   0.006133381277322769,
   0.000866098387632519,
   0.004479038529098034,
   0.006421270780265331,
   -0.001134496065787971,
   0.006671064533293247,
   -0.01478181499

In [18]:
items_to_insert_no_zeros = []
for item_id, item_embedding, item_title in items_to_insert:
    if np.sum(item_embedding) == 0:
        continue
    items_to_insert_no_zeros.append((item_id, item_embedding, item_title))
# items_to_insert = items_to_insert_no_zeros

**Insert items into the index**

In [19]:
from tqdm.auto import tqdm

BATCH_SIZE = 1000

print('Index statistics before upsert:', index.describe_index_stats())

for i in tqdm(range(0, len(items_to_insert_no_zeros), BATCH_SIZE)):
        index.upsert(vectors=items_to_insert_no_zeros[i:i+BATCH_SIZE])

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


  0%|          | 0/50 [00:00<?, ?it/s]

Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48000}},
 'total_vector_count': 48000}


This is a helper method for analysing recommendations later.
This method returns top N products that someone bought in the past (based on product quantity).

In [20]:
def products_bought_by_user_in_the_past(user_id: int, top: int = 10):

    selected = data[data.user_id == user_id].sort_values(by=['total_orders'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products_lookup.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'total_orders']].reset_index(drop=True)
    if selected.shape[0] < top:
        return selected

    return selected[:top]

In [21]:
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


## Query for Recommendations

We are now retrieving user factors for users that we have manually created before for testing purposes. Besides these users, we are adding a random existing user. We are also displaying these users so you can see what these factors look like.

In [22]:
user_ids = [206210, 206211, 103593]
user_factors = model.user_factors[user_to_index[user_ids]]

display(user_factors[1:])

Matrix([[-0.9932781   0.7298836  -0.7355154  -0.3178172  -0.83570546  0.3838572
   1.4104426  -0.53419256 -0.99471194  0.9682674   0.2761275   2.60614
  -0.84276426  0.3325556  -2.100275    1.4665682  -1.8413156   0.9841473
  -1.1957644  -0.46671078  0.6769129   1.1916324   2.0404787   0.37906364
  -0.29733145  0.20436129  0.9765735  -0.0219028  -1.0504111   0.43184435
   0.80593705  1.4691627  -0.29978392  1.2219727  -0.6859106   1.0605284
  -0.02738619 -0.9179155   0.04788572  0.5353661   0.5319488  -1.9663261
  -0.580077   -1.1673372   0.7519745   0.2952755   1.144537    0.9853352
   0.8791097  -2.3855267  -0.6126149   1.1957483   2.6311011   3.3263657
  -1.564149   -2.4432511   0.8510721  -0.89020383 -0.7564133   1.0012891
  -0.40617844 -0.7579703   1.0536174   3.2371724   0.34216052  0.49960852
   0.00758342 -0.873956   -0.5871548  -1.6779717   0.16245282 -1.9881772
   1.1438082   1.0304108   2.4714987  -0.3673678  -1.3453866   0.83291745
  -0.5454309  -0.29746136  0.29281864 -0.3

### Model recommendations

We will now retrieve recommendations from our model directly, just to have these results as a baseline.

In [23]:
print("Model recommendations\n")

start_time = time.process_time()
recommendations0 = model.recommend(userid=user_ids[0], user_items=sparse_user_product[0])
recommendations1 = model.recommend(userid=user_ids[1], user_items=sparse_user_product[1])
recommendations2 = model.recommend(userid=user_ids[2], user_items=sparse_user_product[2])
print("Time needed for retrieving recommended products: " + str(time.process_time() - start_time) + ' seconds.\n')

print('\nRecommendations for person 0:')
for recommendation in recommendations0[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 1:')
for recommendation in recommendations1[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 2:')
for recommendation in recommendations2[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

Model recommendations

Time needed for retrieving recommended products: 0.01539812500000437 seconds.


Recommendations for person 0:
['Sparkling Water']
['Mineral Water']
['Soda']
['Smartwater']
['Spring Water']
['Sparkling Natural Mineral Water']
['Distilled Water']
['Zero Calorie Cola']
['Sparkling Mineral Water']
['Drinking Water']

Recommendations for person 1:
['Large Alfresco Eggs']
['Organic Medium Salsa']
['Organic Spring Mix Salad']
['Pure Irish Butter']
['Organic Spring Mix']
['Baby Wipes Sensitive']
['Chocolate Ice Cream']
['Organic Shredded Carrots']
['No More Tears Baby Shampoo']
['Unsweetened Almond Milk']

Recommendations for person 2:
['Organic Blackberries']
['Organic Blueberries']
['Organic Strawberries']
['Bing Cherries']
['Red Plums']
['Organic Bosc Pear']
['Grape White/Green Seedless']
['Blood Oranges']
['Organic Bartlett Pear']
['Large Yellow Flesh Nectarine']


### Query the index

Let's now query the index to check how quickly we retrieve results. Please note that query speed depends in part on your internet connection.

In [37]:
# Query by user factors
user_embeddings = user_factors.to_numpy()[:-1].tolist() if device == "cuda" else user_factors[:-1].tolist()
print(len(user_embeddings[0]))
start_time = time.process_time()
query_results = [index.query(vector=embedding, top_k=10, include_metadata=True) for embedding in user_embeddings]

print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(user_ids, query_results):
    print(f'user_id={_id}')
    df = pd.DataFrame(
        {
            'products': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_user_in_the_past(_id, top=15))

100
Time needed for retrieving recommended products using Pinecone: 0.07179190000000801 seconds.

user_id=206210
Recommendation: 


,products,scores
0,Mineral Water,0.955505
1,Zero Calorie Cola,0.713828
2,Orange & Lemon Flavor Variety Pack Sparkling F...,0.667592
3,Sparkling Water,0.658520
4,Pain Relief Tablets,0.648211
5,Coppertop AAA Alkaline Batteries,0.633466
6,Milk Chocolate Almonds,0.628515
7,Organic Coconut Water,0.623023
8,Organic Variety Pack,0.620426
9,Extra Fancy Unsalted Mixed Nuts,0.619318


Top buys from the past: 


,product_id,product_name,total_orders
0,22802,Mineral Water,97


user_id=206211
Recommendation: 


,products,scores
0,No More Tears Baby Shampoo,0.973792
1,Size 5 Cruisers Diapers Super Pack,0.655337
2,Baby Bubble Bath,0.645751
3,Head-to-Toe Baby Wash,0.628874
4,Size 6 Baby Dry Diapers,0.599227
5,Cruisers Diapers Jumbo Pack - Size 5,0.597495
6,White Buttermints,0.592676
7,Baby Wipes Sensitive,0.584117
8,Cruisers Size 5 Giant Pack Diapers,0.582672
9,Baby Shampoo With Lavender,0.580210


Top buys from the past: 


,product_id,product_name,total_orders
0,26834,No More Tears Baby Shampoo,89
1,12590,Baby Wash & Shampoo,77


*Note* The inference using Pinecone is much faster compared to retrieving recommendations from a model directly. Please note that this result depends on your internet connection as well. 

All that’s left to do is surface these recommendations on the shopping site, or feed them into other applications.

## Clean up

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.

In [ ]:
pinecone.delete_index(index_name)

## Summary

In this example we used [Pinecone](https://www.pinecone.io/) to build and deploy a product recommendation engine that uses collaborative filtering, relatively quickly.

Once deployed, the product recommendation engine can index new data, retrieve recommendations in milliseconds, and send results to production applications.